In [1]:
import sys
import os
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from docx import Document
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QFileDialog, QLabel
from PyQt5.QtGui import QPainter, QPixmap
from PyQt5.QtCore import Qt
import time


mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

class VideoAnalyzerApp(QWidget):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle('Deltoit Kası Asimetri Analizi')
        self.setGeometry(100, 100, 400, 200)
        
        
        self.background_path = r"C:\Users\monst\Desktop\word\1.png"  # Arka plan resminizin yolu
        self.background_pixmap = QPixmap(self.background_path)

        
        layout = QVBoxLayout()
        
        self.video_path_label = QLabel("Video yolu seçilmedi.")
        layout.addWidget(self.video_path_label)
        
        self.select_video_button = QPushButton('Video Seç')
        self.select_video_button.clicked.connect(self.select_video)
        layout.addWidget(self.select_video_button)
        
        self.select_output_button = QPushButton('Çıktı Klasörü Seç')
        self.select_output_button.clicked.connect(self.select_output_directory)
        layout.addWidget(self.select_output_button)

        self.run_analysis_button = QPushButton('Deltoid Asimetri Ölçümü')  # Button text updated
        self.run_analysis_button.clicked.connect(self.run_analysis)
        layout.addWidget(self.run_analysis_button)
        
        
        self.jump_test_button = QPushButton('Sıçrama Testi')
        self.jump_test_button.clicked.connect(self.run_jump_test)
        layout.addWidget(self.jump_test_button)
        
        self.setLayout(layout)
        
        
        self.output_dir = None
    
    def select_video(self):
        
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.ExistingFiles)
        file_dialog.setNameFilter("Video Dosyaları (*.mp4 *.avi *.mov)")
        
        if file_dialog.exec_():
            selected_files = file_dialog.selectedFiles()
            self.video_path = selected_files[0]
            self.video_path_label.setText(f"Seçilen Video: {self.video_path}")
    
    def select_output_directory(self):
        # Çıktı klasörü seçimi
        folder_dialog = QFileDialog()
        folder_dialog.setFileMode(QFileDialog.Directory)
        
        if folder_dialog.exec_():
            selected_folder = folder_dialog.selectedFiles()
            self.output_dir = selected_folder[0]
            self.video_path_label.setText(f"Çıktı Klasörü: {self.output_dir}")
    
    def run_analysis(self):
        if not hasattr(self, 'video_path') or not self.video_path:
            self.video_path_label.setText("Lütfen bir video seçin.")
            return
        if not self.output_dir:
            self.video_path_label.setText("Lütfen bir çıktı klasörü seçin.")
            return

        video_path = self.video_path
        output_dir = self.output_dir

        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        word_file = os.path.join(output_dir, f"Asimetri_Analizi_{timestamp}.docx")
        graph_path = os.path.join(output_dir, f"Parabolik_Grafik_{timestamp}.png")

        
        doc = Document()

        
        def calculate_angle(a, b, c):
            try:
                ba = a - b
                bc = c - b
                cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                angle = np.arccos(cosine_angle)
                return np.degrees(angle)
            except:
                return 0  

        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Video açılamadı. Lütfen yolun doğru olduğundan emin olun: {video_path}")
            return

        
        time_list = []
        angle_diff_list = []
        start_time = datetime.now()
        frame_count = 0
        fps = cap.get(cv2.CAP_PROP_FPS)
        fps /= 2  

        
        unique_times = set()

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                
                try:
                    right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, 
                                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y])
                    right_elbow = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x, 
                                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y])
                    right_wrist = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x, 
                                            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y])

                    left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, 
                                               landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y])
                    left_elbow = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x, 
                                           landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y])
                    left_wrist = np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x, 
                                           landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y])

                    
                    angle_right = calculate_angle(right_shoulder, right_elbow, right_wrist)
                    angle_left = calculate_angle(left_shoulder, left_elbow, left_wrist)
                    angle_diff = abs(angle_right - angle_left)
                except:
                    angle_diff = 0

                
                elapsed_time = frame_count / fps
                time_list.append(elapsed_time)
                angle_diff_list.append(angle_diff)

               
                asymmetry_side = None
                if 30 < angle_right <= 90 or 30 < angle_left <= 90:
                    if angle_diff > 9:
                        asymmetry_side = "Sağ Kol" if angle_right > angle_left else "Sol Kol"
                elif 90 < angle_right <= 180 or 90 < angle_left <= 180:
                    if angle_diff > 15:
                        asymmetry_side = "Sağ Kol" if angle_right > angle_left else "Sol Kol"

                if asymmetry_side:
                    cv2.putText(image, f"Asimetri: {asymmetry_side}", (10, 50), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                # Çizimler
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            frame_count += 1
            cv2.imshow('Deltoit Kası Analizi', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):  
                break

        cap.release()
        cv2.destroyAllWindows()

        
        plt.figure()
        plt.plot(time_list, angle_diff_list, label="Açı Farkı")
        plt.xlabel("Zaman (s)")
        plt.ylabel("Açı Farkı (°)")
        plt.title("Açı Farklarının Zamanla Değişimi")
        plt.legend()
        plt.savefig(graph_path)

        
        doc.add_heading('Deltoit Kası Asimetri Analizi', level=1)
        doc.add_paragraph(f"Analiz {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} tarihinde yapılmıştır.")
        doc.add_picture(graph_path)
        doc.add_paragraph("Elde edilen sonuçlara göre aşağıdaki değerlere ulaşılmıştır:")

        
        for i, diff in enumerate(angle_diff_list):
            second = int(time_list[i])  
            if diff > 9 or diff > 15:
                if second not in unique_times:
                    unique_times.add(second)  
                    doc.add_paragraph(f"{second} saniyede asimetri tespit edildi.")

        doc.add_paragraph("Grafik ve sonuçlar yukarıda belirtilmiştir.")
        doc.save(word_file)

        print(f"Analiz sonuçları {word_file} konumuna kaydedildi.")

    def run_jump_test(self):
        if not hasattr(self, 'video_path') or not self.video_path:
            self.video_path_label.setText("Lütfen bir video seçin.")
            return
        if not self.output_dir:
            self.video_path_label.setText("Lütfen bir çıktı klasörü seçin.")
            return

        video_path = self.video_path
        output_dir = self.output_dir

       
        cap = cv2.VideoCapture(video_path)

        
        mp_pose = mp.solutions.pose
        pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
        mp_drawing = mp.solutions.drawing_utils

        
        doc = Document()
        doc.add_heading('Zayif Bacak Tespiti Sonuçları', 0)

        
        left_ankle_y = []
        right_ankle_y = []
        time_stamps = []

        
        previous_weak_leg = None

        
        written_seconds = set()

       
        start_time = time.time()  
        last_report_time = start_time  

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = pose.process(rgb_frame)

          
            if result.pose_landmarks:
               
                left_ankle = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]
                right_ankle = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]
                
               
                left_hip = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
                right_hip = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]

              
                left_leg_length = abs(left_ankle.y - left_hip.y)
                right_leg_length = abs(right_ankle.y - right_hip.y)

                
                asimetri_esigi = 0.15  # %15'lik fark

               
                ankle_difference = abs(left_ankle.y - right_ankle.y)

              
                if ankle_difference > (left_leg_length * asimetri_esigi) or ankle_difference > (right_leg_length * asimetri_esigi):
                    cv2.putText(frame, f"Asimetri Tespit Edildi!", (10, 60),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)  # Kırmızı renk
                    
                    
                    current_seconds = int(time.time() - start_time)
                    
                    
                    if current_seconds not in written_seconds:
                        written_seconds.add(current_seconds)
                        # Asimetrik durumu Word belgesine ekle
                        doc.add_paragraph(f"Asimetri Tespit Edildi! Zaman: {current_seconds} saniye - "
                                          f"Sol Bacak Y: {left_ankle.y:.3f}, Sag Bacak Y: {right_ankle.y:.3f}")

               
                left_ankle_y.append(left_ankle.y)
                right_ankle_y.append(right_ankle.y)
                time_stamps.append(time.time() - start_time)

               
                mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                          landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2),
                                          connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2))

            
            cv2.imshow('Pose Estimation', frame)

            
            if cv2.waitKey(1) & 0xFF == ord('q'):  
                break

           
            current_time = time.time()
            if current_time - last_report_time >= 1:
                last_report_time = current_time
                
                seconds = int(current_time - start_time)  

                
                if seconds not in written_seconds:
                    written_seconds.add(seconds)
                    
                    doc.add_paragraph(f'Zaman: {seconds} saniye - Sol Bacak Y: {left_ankle.y:.3f}, Sag Bacak Y: {right_ankle.y:.3f}')

        
        doc.save(os.path.join(output_dir, 'Zayif_Bacak_Tespiti_Sonuclari.docx'))

        
        plt.figure()
        plt.plot(time_stamps, left_ankle_y, label='Sol Bacak Y Koordinatı')
        plt.plot(time_stamps, right_ankle_y, label='Sag Bacak Y Koordinatı')
        plt.xlabel('Zaman (saniye)')
        plt.ylabel('Y Koordinatı')
        plt.title('Sag ve Sol Ayak Bileği Hareketi')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_dir, 'ayak_hareketleri.png'))
        plt.show()

        
        cap.release()
        
        cv2.destroyAllWindows()

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.drawPixmap(0, 0, self.background_pixmap)  # Arka plan resmini pencereye çizin
        super().paintEvent(event)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = VideoAnalyzerApp()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\monst\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
